### Imports

In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import joblib

### Model

In [2]:
class GNN(nn.Module):
    def __init__(self, in_dim=4, hidden=16, out_dim=2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, hidden),
            nn.ReLU(),
            nn.Linear(hidden, out_dim)
        )
    def forward(self, x):
        return self.net(x)

### Load Model

In [3]:
model = GNN()
model.load_state_dict(torch.load("../models/gnn_model.pth", weights_only=True))
model.eval()

scaler = joblib.load("../models/scaler_model.pkl")

### Inputs

In [4]:
edges_input_df = pd.read_csv("../inputs/edges_input.csv")
edges_input = edges_input_df.values.tolist()

X_test = torch.tensor(scaler.transform(edges_input), dtype=torch.float)

### Inference

In [5]:
with torch.no_grad():
    logits = model(X_test)
    probs = F.softmax(logits, dim=1)
    preds = probs.argmax(dim=1)

In [6]:
print(f"{'RSSI':<6} {'ETX':<5} {'Delay':<6} {'Busy':<6} {'Classe':<8} {'Status':<8} {'Confiança':<10}")
print(f"{'-'*55}")

CLASS_NAMES = {0: "Ruim", 1: "Bom"}

for i, row in enumerate(edges_input):
    cls = preds[i].item()
    conf = probs[i, cls].item()
    print(f"{row[0]:<6.0f} {row[1]:<5.1f} {row[2]:<6.0f} {row[3]:<6.2f} "
          f"{cls:<8} {CLASS_NAMES[cls]:<8} {conf:<10.2f}")

RSSI   ETX   Delay  Busy   Classe   Status   Confiança 
-------------------------------------------------------
-60    5.5   78     0.53   0        Ruim     1.00      
-34    2.0   38     0.34   1        Bom      1.00      
-90    6.9   42     0.57   0        Ruim     1.00      
-85    3.2   48     0.59   0        Ruim     1.00      
-45    3.3   12     0.07   1        Bom      1.00      
-90    6.9   86     0.67   0        Ruim     1.00      
-49    2.4   35     0.25   1        Bom      0.99      
-36    1.1   21     0.18   1        Bom      1.00      
-62    5.9   31     0.15   1        Bom      0.71      
-77    8.8   84     0.75   0        Ruim     1.00      
-46    1.7   22     0.26   1        Bom      1.00      
-41    2.7   15     0.24   1        Bom      1.00      
-34    2.5   9      0.20   1        Bom      1.00      
-32    3.0   11     0.17   1        Bom      1.00      
-44    3.6   22     0.13   1        Bom      1.00      
-48    3.6   38     0.10   1        Bom      0.9

### Save

In [7]:
inference_output = pd.DataFrame({
    'class': preds.numpy(),
    'confidence': torch.max(probs, dim=1)[0].numpy()
})

os.makedirs("../outputs", exist_ok=True)
inference_output.to_csv("../outputs/inference_output.csv", index=False)

print("Inferência salva em ../outputs/inference_output.csv")

Inferência salva em ../outputs/inference_output.csv
